In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.environ['USE_PYGEOS'] = '0'
import pysal.lib
import helpers as hs
from importlib import reload
import folium
import pandas as pd
import geopandas as gpd

In [ ]:
%cd ../data

Read geopackage - important to use this instead of read_csv!!!

In [ ]:
pc_gdf = gpd.read_file("pc_gdf.gpkg", layer="cool")

In [ ]:
import r5py
import shapely

In [ ]:
transport_network = r5py.TransportNetwork("petite-couronne.pbf", ["IDFM.gtfs.zip"])

In [ ]:
pc_gdf.columns

In [ ]:
hs.folium_grid_cat_plot(pc_gdf,"restaurant")

In [ ]:
import datetime
import time

In [ ]:
pc_gdf.head()
pc_gdf.rename(columns={'IdINSPIRE':'id'}, inplace = True)

In [ ]:
pc_gdf.columns

In [ ]:
pc_gdf['Log'] = pc_gdf['Log_inc'] + pc_gdf['Log_av45'] + pc_gdf['Log_45_70'] + pc_gdf['Log_70_90'] + pc_gdf['Log_ap90']


In [ ]:
pc_gdf_centered = pc_gdf.copy()
pc_gdf_centered['geometry'] = pc_gdf_centered['geometry'].apply(lambda x : x.centroid)

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=pc_gdf_centered,
    destinations=pc_gdf_centered,
    departure=datetime.datetime(2024,10,8,11,59),
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
)

In [ ]:
travel_times = travel_time_matrix_computer.compute_travel_times()

it took me like max1h to do this!!!

In [ ]:
travel_times.to_csv("travel_times_pc.csv")

In [ ]:
travel_times_square = hs.vert_to_square(travel_times)

In [ ]:
travel_times_square.head()

In [ ]:
reload(hs)

In [ ]:
weights_by_id = hs.transfer_time_to_weight_faster(travel_times_square)

In [ ]:
weights_by_id.head()

In [ ]:
df_reset = weights_by_id.reset_index()

In [ ]:
df_reset.head()

In [ ]:
value = 'CRS3035RES200mN2890000E3760400'
losc = df_reset.where(df_reset == value).stack().index.tolist()
print(losc)

example = pc_gdf[["geometry", "id"]].assign(weight_0 = df_reset.iloc[losc[0][0],1:].values)#
example

In [ ]:
hs.folium_color_map(example, "weight_0")

In [ ]:
pc_gdf = pc_gdf.set_index('id')
pc_gdf.head(2)hs

In [ ]:
weights_by_id = weights_by_id.drop('index', axis=1)
weights_by_id.head()

In [ ]:
weights_by_id = weights_by_id.reset_index()
weights_by_id = weights_by_id.set_index('id')
weights_by_id

In [ ]:
interestVar = ['Log','Log_soc','restaurant',
       'culture and art', 'education', 'food_shops', 'fashion_beauty',
       'supply_shops']
accessibility_measures = hs.calculate_2SFCA_accessibility(pc_gdf,interestVar,weights_by_id)

In [ ]:
for var in interestVar:
    pc_gdf[var+"_access"] = accessibility_measures[var]

In [ ]:
pc_gdf.columns

In [ ]:
pc_gdf.head()

In [ ]:
interestVar2 = ['restaurant',
       'culture and art', 'education', 'food_shops', 'fashion_beauty',
       'supply_shops']
for var in interestVar2:
        pc_gdf[str("weight_" + var)] = pc_gdf[var].sum()/(pc_gdf[interestVar2].sum(axis = 1).sum())

In [ ]:
1 - pc_gdf[[str("weight_" + var) for var in interestVar2]]

## Calculating the aggregated 2SFCA score:

In [ ]:
for var in interestVar2 : 
    pc_gdf[str(var + "_access" + "_norm*weight")] = (pc_gdf[str(var + "_access")].apply(lambda x : (x - pc_gdf[str(var + "_access")].min())/(pc_gdf[str(var + "_access")].max() - pc_gdf[str(var + "_access")].min())))*(1 - pc_gdf[str("weight_" + var)])

In [ ]:
for var in interestVar2 : 
    pc_gdf[str(var + "_access" + "without_norm*weight")] = (pc_gdf[str(var + "_access")].apply(lambda x : (x - pc_gdf[str(var + "_access")].min())/(pc_gdf[str(var + "_access")].max() - 
                                                                                                                                       pc_gdf[str(var + "_access")].min())))

In [ ]:
pc_gdf["CS_aggregated"] = pc_gdf[[i + "_access_norm*weight" for i in interestVar2]].sum(axis = 1) 

In [ ]:
pc_gdf["CS_aggregated_without_weight"] = pc_gdf[[i + "_accesswithout_norm*weight" for i in interestVar2]].sum(axis = 1) 

# Results:

## Without weights:

In [ ]:
reload(hs)

In [ ]:
pc_gdf.columns

In [ ]:
hs.folium_color_map(pc_gdf,"CS_aggregated_without_weight")

## with weights:

In [ ]:
hs.folium_color_map(pc_gdf,"CS_aggregated")

In [ ]:
woods = ["CRS3035RES200mN2891600E3754800","CRS3035RES200mN2891600E3754400","CRS3035RES200mN2891800E3754000","CRS3035RES200mN2892000E3753800",
        "CRS3035RES200mN2891800E3753200","CRS3035RES200mN2891400E3753200","CRS3035RES200mN2891600E3752000","CRS3035RES200mN2891400E3752400",
        "CRS3035RES200mN2891200E3751800","CRS3035RES200mN2891000E3752200","CRS3035RES200mN2890600E3752000","CRS3035RES200mN2890000E3751400",
        "CRS3035RES200mN2890200E3752600","CRS3035RES200mN2889400E3752600","CRS3035RES200mN2890600E3753600","CRS3035RES200mN2890000E3753200",
        "CRS3035RES200mN2889800E3753600","CRS3035RES200mN2890400E3754000","CRS3035RES200mN2890200E3754000","CRS3035RES200mN2893000E3764400",
        "CRS3035RES200mN2887000E3754400","CRS3035RES200mN2887000E3754600","CRS3035RES200mN2887200E3754800","CRS3035RES200mN2887400E3754800",
        "CRS3035RES200mN2885600E3762800","CRS3035RES200mN2885600E3764600","CRS3035RES200mN2886200E3765200","CRS3035RES200mN2886400E3765400",
        "CRS3035RES200mN2885600E3765400","CRS3035RES200mN2886000E3765800","CRS3035RES200mN2886400E3766200","CRS3035RES200mN2886600E3766400",
        "CRS3035RES200mN2886400E3766800","CRS3035RES200mN2886600E3766800","CRS3035RES200mN2886600E3767000","CRS3035RES200mN2886600E3767200",
        "CRS3035RES200mN2886400E3767600","CRS3035RES200mN2886400E3768000","CRS3035RES200mN2886600E3768200","CRS3035RES200mN2886000E3767000",
        "CRS3035RES200mN2886000E3767600","CRS3035RES200mN2885600E3768000","CRS3035RES200mN2885000E3767000","CRS3035RES200mN2885000E3767400",
        "CRS3035RES200mN2885000E3768200","CRS3035RES200mN2884600E3768200","CRS3035RES200mN2884400E3768000"]

In [ ]:
interestVar = ['Log_soc']
clusterVar = [var+"_access" for var in interestVar]#+interestVar
X = pc_gdf[clusterVar]
X = X.drop(woods)
inmur = pc_gdf.drop(woods)

In [ ]:
from pysal.lib import weights

w_queen = weights.contiguity.Queen.from_dataframe(inmur)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import numpy as np
mat = X.values

pipe = Pipeline(steps=[('preprocessor', StandardScaler),
                      ('clustering', AgglomerativeClustering)])

scaler = pipe['preprocessor']()
scaler.fit(X=mat)
mat = scaler.transform(mat)
nb_cluster = 10
clustering = pipe['clustering'](compute_full_tree= True, n_clusters=nb_cluster, connectivity = w_queen.sparse, linkage='ward')
#clustering = pipe['clustering'](n_clusters = nb_cluster,connectivity = w_queen.symmetrize().full()[0])
clustering.fit(mat)
inmur["label"] = clustering.labels_


In [ ]:
hs.folium_color_map(inmur,'label', discrete=True)